In [1]:
import configparser
import sys
from pathlib import Path
from typing import Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sqlalchemy
import sqlite3

# import sqlite_utils

%load_ext sql

In [2]:
sys.path.append(str(Path.cwd().parent / 'src'))

from db_declaration import Base, Record, Artist

In [3]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
plt.style.use('raph-base')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 30)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

np.random.seed(666)

In [4]:
print(sys.executable)
print(sys.version)
print(f'sqlalchemy {sqlalchemy.__version__}')

C:\Users\r2d4\miniconda3\envs\py3\python.exe
3.8.3 (default, May 19 2020, 06:50:17) [MSC v.1916 64 bit (AMD64)]
sqlalchemy 1.3.17


## Connect to DB

In [5]:
Record.__table__

Table('records', MetaData(bind=None), Column('record_id', INTEGER(), table=<records>, primary_key=True, nullable=False), Column('artist_id', INTEGER(), ForeignKey('artists.artist_id'), table=<records>), Column('title', TEXT(), table=<records>, nullable=False), Column('genre_id', INTEGER(), ForeignKey('genres.genre_id'), table=<records>), Column('label', INTEGER(), ForeignKey('labels.label_id'), table=<records>), Column('year', INTEGER(), table=<records>), Column('format_id', INTEGER(), ForeignKey('formats.format_id'), table=<records>), Column('vinyl_color', TEXT(), table=<records>), Column('lim_edition', TEXT(), table=<records>), Column('number', INTEGER(), table=<records>), Column('remarks', TEXT(), table=<records>), Column('purchase_date', TEXT(), table=<records>, nullable=False), Column('price', REAL(), table=<records>, nullable=False), Column('digitized', INTEGER(), table=<records>, nullable=False), Column('rating', INTEGER(), table=<records>), Column('active', INTEGER(), table=<re

In [6]:
rel_path ="db_aka_discobase/Discobase.db"

def create_engine(rel_path: Union[Path, str]) -> sqlalchemy.engine.Engine:
    """Create an engine as factory and pool forthe DB connections."""
    full_path = Path.cwd().parent / rel_path
    print(full_path)
    conn_str = f"sqlite:///{full_path}"
    engine = sqlalchemy.create_engine(conn_str)
    return engine

In [7]:
from sqlalchemy.orm import sessionmaker
# Define a Session class which will serve as a factory for new Session objects
def create_session(engine: sqlalchemy.engine.Engine) -> sqlalchemy.orm.session.Session:
    """Define a Session class bound to the engine
    and instantiate a session object as workspace for
    all ORM operations.
    """
    Session = sessionmaker(bind=engine)
    # Instantiate a session
    session = Session()
    return session

In [8]:
engine = connect_to_db(rel_path)
session = instantiate_session(engine)

C:\Users\r2d4\OneDrive\code\projects\20-02_disco\db_aka_discobase\Discobase.db
<class 'sqlalchemy.orm.session.sessionmaker'>


In [9]:
session.query(Record).all()[0].title

'Ultimate Aggression'

In [18]:
type(session)

sqlalchemy.orm.session.Session

In [11]:
# Connect to DiscoBase.db the sqlite way

db_path = "DiscoBase.db"
conn = sqlite3.connect(db_path, isolation_level=None)  # auto commit mode
cur = conn.cursor()

conn, cur

(<sqlite3.Connection at 0x257535ead50>, <sqlite3.Cursor at 0x257536b8810>)

In [12]:
# Create a connection string and engine for sqlalchemy and ipython-sql

sqlite_conn_str = f"sqlite:///{db_path}"

sqlite_conn_str

'sqlite:///DiscoBase.db'

In [13]:
# Connect to sqlalchemy

engine = sqlalchemy.create_engine(sqlite_conn_str)

engine

Engine(sqlite:///DiscoBase.db)

In [14]:
# Connect to ipython-sql

%sql $sqlite_conn_str

'Connected: @DiscoBase.db'